In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Piauí - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_PI.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego,Piauí - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.649218,24.669760,2.136063e+07,1.358044e+06,6.061141,1.868149e+07,8.192266,21.844
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.649661,25.167145,2.139102e+07,1.359835e+06,6.065065,1.869506e+07,8.186156,15.814
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.650104,25.520404,2.142142e+07,1.361626e+06,6.068988,1.870864e+07,8.180046,16.067
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.650548,25.519725,2.145181e+07,1.363417e+06,6.072912,1.872222e+07,8.173937,14.938
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.650991,25.929312,2.148221e+07,1.365208e+06,6.076835,1.873580e+07,8.167827,17.765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.957215,NaN,NaN,NaN,NaN,NaN,77.776
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.965575,NaN,NaN,NaN,NaN,NaN,72.377
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.977734,NaN,NaN,NaN,NaN,NaN,71.791
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.994123,NaN,NaN,NaN,NaN,NaN,71.015


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.255392,-1.240720,-1.669677,-2.498915,-2.014630,-1.966414,-0.747080
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.215344,-1.193299,-1.652630,-2.431501,-1.986722,-1.939126,-0.751564
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.175296,-1.159620,-1.635584,-2.364087,-1.958813,-1.911838,-0.756047
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.135248,-1.159684,-1.618537,-2.296673,-1.930905,-1.884550,-0.760531
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.095200,-1.120635,-1.601491,-2.229259,-1.902997,-1.857261,-0.765014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.336332,-1.934378,1.184371,0.511529,1.001536,1.022413,1.331534
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.329819,-1.960422,1.174437,0.526506,0.987984,1.008864,1.330230
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.323306,-1.985417,1.164504,0.541484,0.974431,0.995315,1.328926
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.316792,-2.009327,1.154570,0.556461,0.960879,0.981765,1.327622


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      18.413
1      16.605
2      22.084
3      22.445
4      23.323
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Piauí - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.255392,-1.240720,-1.669677,-2.498915,-2.014630,-1.966414,-0.747080
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.215344,-1.193299,-1.652630,-2.431501,-1.986722,-1.939126,-0.751564
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.175296,-1.159620,-1.635584,-2.364087,-1.958813,-1.911838,-0.756047
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.135248,-1.159684,-1.618537,-2.296673,-1.930905,-1.884550,-0.760531
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.095200,-1.120635,-1.601491,-2.229259,-1.902997,-1.857261,-0.765014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.382405,-0.210180,1.192759,0.339852,1.041980,1.070621,1.460989
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.385453,-0.282985,1.199072,0.340094,1.049362,1.077532,1.459107
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.388501,-0.356527,1.205385,0.340336,1.056744,1.084444,1.457225
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.391549,-0.430817,1.211698,0.340578,1.064126,1.091356,1.455343


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      18.413
1      16.605
2      22.084
3      22.445
4      23.323
        ...  
157    54.656
158    69.984
159    58.643
160    72.337
161    70.968
Name: Piauí - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 14)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - IDH,Piauí - Produção de Cimento (t),Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - Desemprego
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.390843,1.086761,0.792827,1.242816,0.868027,0.870142,0.720289
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.390549,1.038665,0.813735,1.251507,0.885512,0.888110,0.759067
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.390254,0.991577,0.834643,1.260198,0.902996,0.906077,0.797846
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.389960,0.947395,0.855550,1.268889,0.920480,0.924044,0.836625
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.389666,0.907255,0.876458,1.277580,0.937965,0.942011,0.875403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.336332,-1.934378,1.184371,0.511529,1.001536,1.022413,1.331534
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.329819,-1.960422,1.174437,0.526506,0.987984,1.008864,1.330230
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.323306,-1.985417,1.164504,0.541484,0.974431,0.995315,1.328926
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.316792,-2.009327,1.154570,0.556461,0.960879,0.981765,1.327622


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 14)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=True), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[958969159, 1520770289, 3331761389, 551048648, 3509437564, 1292704911, 1094386321, 934353967, 183108740, 418180191, 616127082, 3727514219, 607364905, 3198369509, 1149960915, 774296531, 2067565654, 4079437139, 79457486, 3388771432, 4262720829, 879383959, 158802365, 1133625406, 2085045815, 1180403099, 3625883426, 1475759085, 3959724169, 3100720361, 2519128322, 2568799440, 3897922564, 3981323384, 3948489216, 2450243045, 4041008022, 1107677724, 838039801, 1209989959, 67592475, 2082618837, 307666975, 3767586797, 4007800168, 2046604468, 3240723585, 2203778629, 1093220443, 3556096554]


Step: 0 ___________________________________________
val_loss: 8.26579761505127
winner_seed: 958969159


Step: 1 ___________________________________________
val_loss: 9.96822738647461


Step: 2 ___________________________________________
val_loss: 8.63857650756836


Step: 3 ___________________________________________
val_loss: 12.571648597717285


Step: 4 ___________________________________________
val_loss: 7.

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 213ms/step - loss: 2705.2729 - val_loss: 632.6850
Epoch 2/10000
4/4 [==============================] - 0s 14ms/step - loss: 631.9100 - val_loss: 258.4655
Epoch 3/10000
4/4 [==============================] - 0s 13ms/step - loss: 229.9429 - val_loss: 158.4794
Epoch 4/10000
4/4 [==============================] - 0s 15ms/step - loss: 410.1641 - val_loss: 99.8061
Epoch 5/10000
4/4 [==============================] - 0s 12ms/step - loss: 178.0396 - val_loss: 54.9470
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 70.8991 - val_loss: 41.3742
Epoch 7/10000
4/4 [==============================] - 0s 12ms/step - loss: 63.6067 - val_loss: 44.7321
Epoch 8/10000
4/4 [==============================] - 0s 12ms/step - loss: 46.8413 - val_loss: 67.8954
Epoch 9/10000
4/4 [==============================] - 0s 13ms/step - loss: 101.9152 - val_loss: 69.5631
Epoch 10/10000
4/4 [==============================] - 0s 12ms/step - lo

4/4 [==============================] - 0s 13ms/step - loss: 28.7972 - val_loss: 34.6345
Epoch 81/10000
4/4 [==============================] - 0s 13ms/step - loss: 30.1498 - val_loss: 30.8439
Epoch 82/10000
4/4 [==============================] - 0s 12ms/step - loss: 29.1618 - val_loss: 69.2395
Epoch 83/10000
4/4 [==============================] - 0s 13ms/step - loss: 31.4840 - val_loss: 36.7131
Epoch 84/10000
4/4 [==============================] - 0s 13ms/step - loss: 33.0731 - val_loss: 36.9324
Epoch 85/10000
4/4 [==============================] - 0s 12ms/step - loss: 28.5597 - val_loss: 39.0879
Epoch 86/10000
4/4 [==============================] - 0s 11ms/step - loss: 29.7850 - val_loss: 31.6584
Epoch 87/10000
4/4 [==============================] - 0s 13ms/step - loss: 31.5068 - val_loss: 43.8634
Epoch 88/10000
4/4 [==============================] - 0s 13ms/step - loss: 46.1104 - val_loss: 91.7321
Epoch 89/10000
4/4 [==============================] - 0s 12ms/step - loss: 41.5474 - val

4/4 [==============================] - 0s 13ms/step - loss: 23.5243 - val_loss: 29.8071
Epoch 160/10000
4/4 [==============================] - 0s 13ms/step - loss: 25.6976 - val_loss: 33.1502
Epoch 161/10000
4/4 [==============================] - 0s 12ms/step - loss: 27.4998 - val_loss: 30.3290
Epoch 162/10000
4/4 [==============================] - 0s 12ms/step - loss: 22.3619 - val_loss: 33.5306
Epoch 163/10000
4/4 [==============================] - 0s 12ms/step - loss: 27.2433 - val_loss: 25.5349
Epoch 164/10000
4/4 [==============================] - 0s 13ms/step - loss: 23.3277 - val_loss: 38.1149
Epoch 165/10000
4/4 [==============================] - 0s 13ms/step - loss: 25.8791 - val_loss: 26.5729
Epoch 166/10000
4/4 [==============================] - 0s 13ms/step - loss: 26.3438 - val_loss: 34.8734
Epoch 167/10000
4/4 [==============================] - 0s 13ms/step - loss: 25.3885 - val_loss: 41.4308
Epoch 168/10000
4/4 [==============================] - 0s 13ms/step - loss: 24.9

Epoch 238/10000
4/4 [==============================] - 0s 12ms/step - loss: 18.6796 - val_loss: 25.1509
Epoch 239/10000
4/4 [==============================] - 0s 12ms/step - loss: 21.2582 - val_loss: 29.6837
Epoch 240/10000
4/4 [==============================] - 0s 12ms/step - loss: 22.0747 - val_loss: 32.3530
Epoch 241/10000
4/4 [==============================] - 0s 13ms/step - loss: 22.0869 - val_loss: 39.9714
Epoch 242/10000
4/4 [==============================] - 0s 14ms/step - loss: 22.0782 - val_loss: 27.4987
Epoch 243/10000
4/4 [==============================] - 0s 13ms/step - loss: 26.3611 - val_loss: 37.6172
Epoch 244/10000
4/4 [==============================] - 0s 13ms/step - loss: 24.2643 - val_loss: 30.8478
Epoch 245/10000
4/4 [==============================] - 0s 12ms/step - loss: 20.8576 - val_loss: 37.5381
Epoch 246/10000
4/4 [==============================] - 0s 12ms/step - loss: 19.7518 - val_loss: 21.4334
Epoch 247/10000
4/4 [==============================] - 0s 12ms/s

Epoch 317/10000
4/4 [==============================] - 0s 12ms/step - loss: 15.0044 - val_loss: 27.6636
Epoch 318/10000
4/4 [==============================] - 0s 14ms/step - loss: 13.8229 - val_loss: 31.4659
Epoch 319/10000
4/4 [==============================] - 0s 14ms/step - loss: 12.0717 - val_loss: 25.6041
Epoch 320/10000
4/4 [==============================] - 0s 14ms/step - loss: 14.0056 - val_loss: 24.8004
Epoch 321/10000
4/4 [==============================] - 0s 14ms/step - loss: 16.6046 - val_loss: 24.6330
Epoch 322/10000
4/4 [==============================] - 0s 14ms/step - loss: 15.7754 - val_loss: 19.6143
Epoch 323/10000
4/4 [==============================] - 0s 14ms/step - loss: 13.8823 - val_loss: 29.1128
Epoch 324/10000
4/4 [==============================] - 0s 13ms/step - loss: 15.3458 - val_loss: 33.8597
Epoch 325/10000
4/4 [==============================] - 0s 12ms/step - loss: 16.4866 - val_loss: 22.3085
Epoch 326/10000
4/4 [==============================] - 0s 13ms/s

...

4/4 [==============================] - 0s 10ms/step - loss: 9.5153 - val_loss: 16.9368
Epoch 871/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.7370 - val_loss: 22.2705
Epoch 872/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.2842 - val_loss: 21.9085
Epoch 873/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.1225 - val_loss: 19.0558
Epoch 874/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.2589 - val_loss: 31.9039
Epoch 875/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.6614 - val_loss: 20.9230
Epoch 876/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.0934 - val_loss: 35.4618
Epoch 877/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.8696 - val_loss: 22.9967
Epoch 878/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.3467 - val_loss: 26.0028
Epoch 879/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.7541 - val_

4/4 [==============================] - 0s 13ms/step - loss: 6.9128 - val_loss: 14.0766
Epoch 950/10000
4/4 [==============================] - 0s 13ms/step - loss: 9.9898 - val_loss: 26.1325
Epoch 951/10000
4/4 [==============================] - 0s 14ms/step - loss: 9.9150 - val_loss: 18.9618
Epoch 952/10000
4/4 [==============================] - 0s 13ms/step - loss: 6.5353 - val_loss: 12.6396
Epoch 953/10000
4/4 [==============================] - 0s 14ms/step - loss: 8.1670 - val_loss: 31.0658
Epoch 954/10000
4/4 [==============================] - 0s 13ms/step - loss: 6.2719 - val_loss: 27.0508
Epoch 955/10000
4/4 [==============================] - 0s 13ms/step - loss: 6.7034 - val_loss: 20.3442
Epoch 956/10000
4/4 [==============================] - 0s 13ms/step - loss: 7.6414 - val_loss: 20.7673
Epoch 957/10000
4/4 [==============================] - 0s 14ms/step - loss: 7.6384 - val_loss: 21.6856
Epoch 958/10000
4/4 [==============================] - 0s 12ms/step - loss: 8.4443 - val_

4/4 [==============================] - 0s 12ms/step - loss: 6.4363 - val_loss: 32.8414
Epoch 1029/10000
4/4 [==============================] - 0s 12ms/step - loss: 6.2741 - val_loss: 16.6877
Epoch 1030/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.6538 - val_loss: 29.1344
Epoch 1031/10000
4/4 [==============================] - 0s 12ms/step - loss: 7.1627 - val_loss: 26.1364
Epoch 1032/10000
4/4 [==============================] - 0s 12ms/step - loss: 6.5027 - val_loss: 22.9909
Epoch 1033/10000
4/4 [==============================] - 0s 12ms/step - loss: 9.7015 - val_loss: 23.7245
Epoch 1034/10000
4/4 [==============================] - 0s 12ms/step - loss: 5.7343 - val_loss: 26.0032
Epoch 1035/10000
4/4 [==============================] - 0s 12ms/step - loss: 6.6121 - val_loss: 21.3173
Epoch 1036/10000
4/4 [==============================] - 0s 13ms/step - loss: 6.0671 - val_loss: 27.0174
Epoch 1037/10000
4/4 [==============================] - 0s 12ms/step - loss: 6.25

Epoch 1107/10000
4/4 [==============================] - 0s 12ms/step - loss: 6.4336 - val_loss: 23.1783
Epoch 1108/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.5397 - val_loss: 16.8555
Epoch 1109/10000
4/4 [==============================] - 0s 14ms/step - loss: 5.9922 - val_loss: 14.5509
Epoch 1110/10000
4/4 [==============================] - 0s 15ms/step - loss: 7.8377 - val_loss: 16.2547
Epoch 1111/10000
4/4 [==============================] - 0s 13ms/step - loss: 7.8491 - val_loss: 19.0158
Epoch 1112/10000
4/4 [==============================] - 0s 13ms/step - loss: 6.7737 - val_loss: 20.2332
Epoch 1113/10000
4/4 [==============================] - 0s 12ms/step - loss: 7.3783 - val_loss: 16.5271
Epoch 1114/10000
4/4 [==============================] - 0s 12ms/step - loss: 6.8416 - val_loss: 24.0156
Epoch 1115/10000
4/4 [==============================] - 0s 13ms/step - loss: 11.7733 - val_loss: 36.6126
Epoch 1116/10000
4/4 [==============================] - 0s 12ms

Epoch 1186/10000
4/4 [==============================] - 0s 13ms/step - loss: 6.5189 - val_loss: 27.0721
Epoch 1187/10000
4/4 [==============================] - 0s 13ms/step - loss: 6.0703 - val_loss: 17.2312
Epoch 1188/10000
4/4 [==============================] - 0s 12ms/step - loss: 5.5815 - val_loss: 22.3300
Epoch 1189/10000
4/4 [==============================] - 0s 13ms/step - loss: 5.2364 - val_loss: 23.2492
Epoch 1190/10000
4/4 [==============================] - 0s 13ms/step - loss: 6.0491 - val_loss: 33.6576
Epoch 1191/10000
4/4 [==============================] - 0s 13ms/step - loss: 5.1017 - val_loss: 21.1404
Epoch 1192/10000
4/4 [==============================] - 0s 13ms/step - loss: 4.3362 - val_loss: 21.9812
Epoch 1193/10000
4/4 [==============================] - 0s 13ms/step - loss: 5.2119 - val_loss: 25.8875
Epoch 1194/10000
4/4 [==============================] - 0s 12ms/step - loss: 5.3363 - val_loss: 22.5475
Epoch 1195/10000
4/4 [==============================] - 0s 12ms/

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,66.059906,66.105484,66.033966,65.851639,65.788116,65.781769,65.806961,65.687454,65.564842,65.329613,65.171692,65.130974,64.963989,64.861496,64.859276,64.67234,64.269516,63.797886,63.86203,64.017075,63.528534,62.883827,62.583447,62.624809,62.604527,60.969688,57.614059,56.055195,55.625744,62.891449
Target,75.913,74.774,72.484,71.534,70.32,67.364,59.688,40.259,41.239,45.035,42.477,57.619,53.109,63.141,55.58,60.001,62.767,45.308,57.142,46.283,43.39,40.824,55.063,56.474,68.133,68.642,64.741,71.269,67.631,62.845
Error,9.853096,8.668518,6.450035,5.682358,4.531883,1.58223,6.118961,25.428455,24.325844,20.294613,22.694691,7.511974,11.854988,1.720497,9.279274,4.671341,1.502518,18.489887,6.720032,17.734074,20.138535,22.059826,7.520447,6.15081,5.528477,7.67231,7.126938,15.213802,12.005253,0.046448


In [38]:
display(mae)
display(mape)

10.619271

0.21047232

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[788.3124]] - Target[718.706]| =  Error: [[69.606384]]; MAPE:[[0.0968496]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[766.92426]] - Target[639.082]| =  Error: [[127.842285]]; MAPE:[[0.20004052]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[355.76065]] - Target[403.26099999999997]| =  Error: [[47.500336]]; MAPE:[[0.11779056]]


[array([[69.606384]], dtype=float32),
 array([[127.842285]], dtype=float32),
 array([[47.500336]], dtype=float32)]

81.649666

0.13822688